Obtain statistics on BMLP_GPU runtime

In [11]:
import cupy as cp 
import numpy as np
from bmlp.core import matrix

# Check if connected to GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU: need to configure runtime type to an environment with an accessible GPU')
else:
    print('Connected to a GPU - GPU info summary: \n\n' + gpu_info)

Connected to a GPU - GPU info summary: 

Wed Mar  5 01:00:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.41                 Driver Version: 561.03         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  |   00000000:01:00.0  On |                  N/A |
| 32%   27C    P8              5W /  160W |     772MiB /   8188MiB |      4%      Default |
|                                         |                        |                  N/A |
+------

In [1]:
# Use python wrapper of GraphBLAS on GPU (BLAS - Basic Linear Algebra Subprograms)
# GraphBLAS supports graph operations via linear algebraic methods (e.g. matrix multiplication) over various semirings

# GraphBLAS version of BMLP-RMS algorithm which performs repeated matrix squaring
from bmlp.core.matrix import *

In [2]:
# Simple graph example 1
# Create edge list (directed graph) and the number of nodes (matrix dimension)
edges = [(0, 1), (1, 2), (2, 3), (3, 4)]
num_nodes = 5

In [3]:
# Create a square adjacency matrix using BOOL type
R1 = new(num_nodes, num_nodes)

# Insert edges into the adjacency matrix
for src, dst in edges:
    R1[src, dst] = True
print('R1 = \n'+ str(R1) + '\n')
        
# Print the transitive closure matrix
print('closure R2* = \n' + str(RMS(R1, print_matrix=True)) + '\n')

R1 = 
"M_0"      nvals  nrows  ncols  dtype         format
gb.Matrix      4      5      5   BOOL  bitmapr (iso)
----------------------------------------------------
  0     1     2     3     4
0    True                  
1          True            
2                True      
3                      True
4                          

R = R2 + I = 
"M_2"      nvals  nrows  ncols  dtype         format
gb.Matrix      9      5      5   BOOL  bitmapr (iso)
----------------------------------------------------
      0     1     2     3     4
0  True  True                  
1        True  True            
2              True  True      
3                    True  True
4                          True

fixpoint = 
"M_3"      nvals  nrows  ncols  dtype         format
gb.Matrix     12      5      5   BOOL  bitmapr (iso)
----------------------------------------------------
      0     1     2     3     4
0  True  True  True            
1        True  True  True      
2              True  True  True
3

In [4]:
# Simple graph example 2
# Create edge list (directed graph) and the number of nodes (matrix dimension)
edges = [(0, 1), (1, 2), (2, 3), (3, 0), (4, 4)]
num_nodes = 5

In [5]:
# Create a square adjacency matrix using BOOL type
R1 = new(num_nodes, num_nodes)

# Insert edges into the adjacency matrix
for src, dst in edges:
    R1[src, dst] = True
print('R1 = \n'+ str(R1) + '\n')
        
# Print the transitive closure matrix
print('closure R2* = \n' + str(RMS(R1)) + '\n')

R1 = 
"M_5"      nvals  nrows  ncols  dtype         format
gb.Matrix      5      5      5   BOOL  bitmapr (iso)
----------------------------------------------------
      0     1     2     3     4
0        True                  
1              True            
2                    True      
3  True                        
4                          True

closure R2* = 
"M_9"      nvals  nrows  ncols  dtype         format
gb.Matrix     17      5      5   BOOL  bitmapr (iso)
----------------------------------------------------
      0     1     2     3     4
0  True  True  True  True      
1  True  True  True  True      
2  True  True  True  True      
3  True  True  True  True      
4                          True



In [6]:
# Create a vector to represent a query
V = new(num_nodes)

# query the reachability of node 3
V[3] = True

print('closure V* = \n' + str(SMP(V,R1)) + '\n')

closure V* = 
"v_7"      nvals  size  dtype        format
gb.Vector      4     5   BOOL  bitmap (iso)
-------------------------------------------
index     0     1     2     3 4
value  True  True  True  True  



In [7]:
R1 = new(3, 3)
R1[0,0] = True
R1[0,1] = True
R1[1,2] = True
# R1 = R1.apply(gb.types.BOOL.LNOT)
print(R1)

print(negate(R1))


"M_10"     nvals  nrows  ncols  dtype         format
gb.Matrix      3      3      3   BOOL  bitmapr (iso)
----------------------------------------------------
      0     1     2
0  True  True      
1              True
2                  
"M_14"     nvals  nrows  ncols  dtype         format
gb.Matrix      6      3      3   BOOL  bitmapr (iso)
----------------------------------------------------
      0     1     2
0              True
1  True  True      
2  True  True  True


In [8]:
import random, time

# Complex graph
num_nodes = 5000
all_edges = [(i, j) for i in range(num_nodes) for j in range(num_nodes)]
print('No. edges: ' + str(len(all_edges)))

No. edges: 25000000


In [ ]:
# num_reps = 10
# p = 0.5
# total_time = []

# # repeat num_reps times
# for i in range(num_reps):

#     # sample edges with edge probability < p
#     sampled_edges = list(filter(lambda _: random.random() < p, all_edges))

#     # Create a square adjacency matrix using BOOL type
#     empty_matrix = new(gb.BOOL, num_nodes, num_nodes)
#     R1 = empty_matrix

#     # Insert edges into the adjacency matrix
#     for src, dst in sampled_edges:
#         R1[src, dst] = True

#     # Run and time the BMLP-RMS module
#     start_time = time.time()
#     RMS(R1)
#     end_time = time.time()

#     total_time.append(end_time - start_time)
#     print('Wall time: ' + str(end_time - start_time))

# print('Mean wall time: ' + str(np.mean(total_time)))
# print('Wall time sterr: ' + str(np.std(total_time) / np.sqrt(len(total_time))))


In [10]:
num_reps = 10
p = 0.5
total_time = []

# repeat num_reps times
for i in range(num_reps):

    # sample edges with edge probability < p
    sampled_edges = list(filter(lambda _: random.random() < p, all_edges))

    # Create a square adjacency matrix using BOOL type
    empty_matrix = new(num_nodes, num_nodes)
    R1 = empty_matrix

    # Insert edges into the adjacency matrix
    for src, dst in sampled_edges:
        R1[src, dst] = True
        
    # Create a vector to represent a query
    V = new(num_nodes)

    # query the reachability of node 3
    V[1] = True

    # Run and time the BMLP-SMP module
    start_time = time.time()
    SMP(V,R1)
    end_time = time.time()

    total_time.append(end_time - start_time)
    print('Wall time: ' + str(end_time - start_time))

print('Mean wall time: ' + str(np.mean(total_time)))
print('Wall time sterr: ' + str(np.std(total_time) / np.sqrt(len(total_time))))

Wall time: 0.04276418685913086
Wall time: 0.024487733840942383
Wall time: 0.032889604568481445
Wall time: 0.024493694305419922
Wall time: 0.03745698928833008
Wall time: 0.0311434268951416
Wall time: 0.03785300254821777
Wall time: 0.029917240142822266
Wall time: 0.029431581497192383
Wall time: 0.03070855140686035
Mean wall time: 0.032114601135253905
Wall time sterr: 0.001751225682066303
